# Data preparation

NeuralGCM models take and produce data on defined on [37 pressure levels](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels), including the following variables, provided in SI units and on the NeuralGCM model's native grid:

- Inputs/outputs (on pressure levels): `u_component_of_wind`, `v_component_of_wind`, `geopotential`, `temperature`, `specific_humidity`, `specific_cloud_ice_water_content`, `specific_cloud_liquid_water_content`.
- Forcings (surface level only): `sea_surface_temperature`, `sea_ice_cover`

## Regridding data

Preparing a dataset stored on a different horizontal grid for NeuralGCM requires two steps:

1. Horizontal regridding to a Gaussian grid. For processing fine-resolution data conservative regridding is most appropriate (and is what we used to train NeuralGCM).
2. Filling in all missing values (NaN), to ensure all inputs are valid. Forcing fields like `sea_surface_temperature` are only defined over ocean in ERA5, and NeuralGCM's surface model also includes a mask that ignores values over land, but we still need to fill all NaN values to them leaking into our model outputs.

Utilities for both of these operations are packaged as part of Dinosaur. We'll show how to use them on the Zarr copy of ERA5 from the [ARCO-ERA5](https://github.com/google-research/arco-era5) project:

In [1]:
import numpy as np
import gcsfs
import xarray

# create a xarray.Dataset with required variables for NeuralGCM
gcs = gcsfs.GCSFileSystem(token='anon')
# Pythonic file-system for Google Cloud Storage

path = 'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3'

full_era5 = xarray.open_zarr(gcs.get_mapper(path), chunks=None)

In [2]:
full_era5

<xarray.Dataset> Size: 4PB
Dimensions:                                                          (
                                                                      time: 1323648,
                                                                      latitude: 721,
                                                                      longitude: 1440,
                                                                      level: 37)
Coordinates:
  * latitude                                                         (latitude) float32 3kB ...
  * level                                                            (level) int64 296B ...
  * longitude                                                        (longitude) float32 6kB ...
  * time                                                             (time) datetime64[ns] 11MB ...
Data variables: (12/273)
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    100m_v_component_of_wind                                         (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB ...
    10m_u_component_of_wind                                          (time, latitude, longitude) float32 5TB ...
    10m_v_component_of_neutral_wind                                  (time, latitude, longitude) float32 5TB ...
    10m_v_component_of_wind                                          (time, latitude, longitude) float32 5TB ...
    ...                                                               ...
    wave_spectral_directional_width_for_swell                        (time, latitude, longitude) float32 5TB ...
    wave_spectral_directional_width_for_wind_waves                   (time, latitude, longitude) float32 5TB ...
    wave_spectral_kurtosis                                           (time, latitude, longitude) float32 5TB ...
    wave_spectral_peakedness                                         (time, latitude, longitude) float32 5TB ...
    wave_spectral_skewness                                           (time, latitude, longitude) float32 5TB ...
    zero_degree_level                                                (time, latitude, longitude) float32 5TB ...
Attributes:
    valid_time_start:       1940-01-01
    last_updated:           2025-02-17 10:43:25.472451+00:00
    valid_time_stop:        2024-11-30
    valid_time_stop-era5t:  ['2025-02-11']

In [3]:
level_1000 = full_era5.sel(level=1000, latitude=40.75, longitude=286, time=slice('2022-01-01', '2022-12-31'))

In [4]:
selected_vars = level_1000[['temperature', 'specific_humidity', 'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content']]
selected_vars['sea_surface_temperature'] = level_1000['sea_surface_temperature']
selected_vars['sea_ice_cover'] = level_1000['sea_ice_cover']
selected_vars

<xarray.Dataset> Size: 280kB
Dimensions:                              (time: 8760)
Coordinates:
    latitude                             float32 4B 40.75
    level                                int64 8B 1000
    longitude                            float32 4B 286.0
  * time                                 (time) datetime64[ns] 70kB 2022-01-0...
Data variables:
    temperature                          (time) float32 35kB ...
    specific_humidity                    (time) float32 35kB ...
    specific_cloud_ice_water_content     (time) float32 35kB ...
    specific_cloud_liquid_water_content  (time) float32 35kB ...
    sea_surface_temperature              (time) float32 35kB ...
    sea_ice_cover                        (time) float32 35kB ...
Attributes:
    valid_time_start:       1940-01-01
    last_updated:           2025-02-17 10:43:25.472451+00:00
    valid_time_stop:        2024-11-30
    valid_time_stop-era5t:  ['2025-02-11']

In [ ]:
selected_vars.to_netcdf('8518750_ERA5.nc')